# Final Project
Clay Kindiger and Haoyang Pang

## Load Libraries

In [114]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import math
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

## Load Original Data

In [23]:
players_team = pd.read_csv('full_player_stats.csv')
advanced_stats = pd.read_csv('data/advanced_players.csv')
mvp_voting = pd.read_csv('data/mvp_voting_shares.csv')
dpoy_voting = pd.read_csv('data/dpoy_voting.csv')

In [3]:
players_team.head()

,Season,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,L,W/L%,GB,PS/G,PA/G,SRS,Playoffs,Losing_season,mvp,dpoy
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,0.0,530.0,103.0,273.0,...,55,0.329,34.0,93.1,98.7,-5.83,0,1,0,0
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,16.0,959.0,144.0,357.0,...,55,0.329,34.0,93.1,98.7,-5.83,0,1,0,0
2,1998,Terry Dehere,PG,26.0,SAC,77.0,18.0,1410.0,180.0,451.0,...,55,0.329,34.0,93.1,98.7,-5.83,0,1,0,0
3,1998,Lawrence Funderburke,PF,27.0,SAC,52.0,1.0,1094.0,191.0,390.0,...,55,0.329,34.0,93.1,98.7,-5.83,0,1,0,0
4,1998,Derek Grimm,SF,23.0,SAC,9.0,0.0,34.0,4.0,14.0,...,55,0.329,34.0,93.1,98.7,-5.83,0,1,0,0


In [4]:
advanced_stats.head()

,Season,Player,Pos,Age,Tm,G,MP,PER,TS%,3PAr,...,Unnamed: 19,OWS,DWS,WS,WS/48,Unnamed: 24,OBPM,DBPM,BPM,VORP
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,530.0,10.5,0.405,0.114,...,NaN,-0.5,0.2,-0.3,-0.025,NaN,-3.2,-1.5,-4.7,-0.4
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,959.0,10.1,0.456,0.053,...,NaN,-0.8,0.6,-0.2,-0.008,NaN,-3.6,-1.2,-4.8,-0.7
2,1998,Shareef Abdur-Rahim,SF,21.0,VAN,82.0,2950.0,21.1,0.562,0.038,...,NaN,6.3,0.6,6.9,0.113,NaN,3.4,-1.8,1.6,2.6
3,1998,Cory Alexander,PG,24.0,TOT,60.0,1298.0,15.2,0.548,0.44,...,NaN,1.0,1.2,2.2,0.081,NaN,0.4,0.1,0.5,0.8
4,1998,Cory Alexander,PG,24.0,SAS,37.0,501.0,11.1,0.512,0.441,...,NaN,-0.2,0.8,0.7,0.065,NaN,-2.3,1.1,-1.2,0.1


In [5]:
mvp_voting.head()

,Season,Player,Pts Won,Pts Max,Share
0,1998,Michael Jordan,1084.0,1160,0.934
1,1998,Karl Malone,842.0,1160,0.726
2,1998,Gary Payton,431.0,1160,0.372
3,1998,Shaquille O'Neal,311.0,1160,0.268
4,1998,Tim Duncan,148.0,1160,0.128


In [6]:
dpoy_voting.head()

,Rank,Player,Age,Tm,First,Pts Won,Pts Max,Share,G,MP,...,BLK,FG%,3P%,FT%,WS,WS/48,DWS,DBPM,DRtg,-9999
0,1,Marcus Smart,27,BOS,37.0,257.0,500,0.514,71,32.3,...,0.3,0.418,0.331,0.793,5.6,0.116,3.7,1.7,107,smartma01
1,2,Mikal Bridges,25,PHO,22.0,202.0,500,0.404,82,34.8,...,0.4,0.534,0.369,0.834,8.9,0.150,3.7,1.1,110,bridgmi01
2,3,Rudy Gobert,29,UTA,12.0,136.0,500,0.272,66,32.1,...,2.1,0.713,0.000,0.690,11.7,0.264,4.3,1.7,103,goberru01
3,4,Bam Adebayo,24,MIA,13.0,128.0,500,0.256,56,32.6,...,0.8,0.557,0.000,0.753,7.2,0.188,3.5,2.1,104,adebaba01
4,5,Jaren Jackson Jr.,22,MEM,10.0,99.0,500,0.198,78,27.3,...,2.3,0.415,0.319,0.823,5.4,0.121,3.7,0.8,106,jacksja02


## Clean Data

In [26]:
players_team.drop(columns = ['mvp','dpoy'],inplace = True)

In [57]:
season = []
start = 2023
for idx,row in dpoy_voting.iterrows():
    if row['Rank'] == '1':
        start -= 1
    
    season.append(start)
dpoy_voting['Season'] = season

In [58]:
dpoy_voting = dpoy_voting[['Season', 'Player',  'Pts Won', 'Pts Max', 'Share']]
dpoy_voting.head()

,Season,Player,Pts Won,Pts Max,Share
0,2022,Marcus Smart,257.0,500,0.514
1,2022,Mikal Bridges,202.0,500,0.404
2,2022,Rudy Gobert,136.0,500,0.272
3,2022,Bam Adebayo,128.0,500,0.256
4,2022,Jaren Jackson Jr.,99.0,500,0.198


In [16]:
advanced_stats = advanced_stats[advanced_stats['Player'] != 'Player']

In [24]:
players_teams = {}
for ply,group in advanced_stats.groupby(['Player','Season']):
    if group.shape[0] > 1:
        tm = list(group.Tm)[-1]
        if 28 in group.index:
            print(group.Tm)
        players_teams[list(group.index)[0]] = tm
        if tm == 'TOT':
            print(ply)

28     Tm
57     Tm
82     Tm
105    Tm
130    Tm
155    Tm
178    Tm
201    Tm
222    Tm
247    Tm
270    Tm
291    Tm
316    Tm
339    Tm
364    Tm
387    Tm
416    Tm
453    Tm
480    Tm
517    Tm
544    Tm
Name: Tm, dtype: object


In [25]:
teams_list = []
for idx,row in advanced_stats.iterrows():
    if idx in players_teams.keys():
        tm = players_teams[idx]
        teams_list.append(tm)
    else:
        tm = row['Tm']
        if idx == 3613:
            tm = 'UTA'
        elif idx == 21:
            tm = 'MIA'
        elif idx == 4189:
            tm = 'WAS'
        elif idx == 3037:
            tm = 'BOS'
        elif idx == 11099:
            tm = 'CHI'
        elif idx == 6128:
            tm = 'NJN'
        teams_list.append(tm)
advanced_stats['Tm'] = teams_list

In [46]:
advanced_stats.drop_duplicates(['Player','Season'],keep = 'first',inplace = True)

In [28]:
players_team.shape

(11862, 41)

In [31]:
advanced_stats.shape

(15377, 29)

In [33]:
advanced_stats.columns

Index(['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'MP', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'Unnamed: 19', 'OWS', 'DWS', 'WS', 'WS/48', 'Unnamed: 24', 'OBPM',
       'DBPM', 'BPM', 'VORP'],
      dtype='object')

In [34]:
players_team.columns

Index(['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Year', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS',
       'Playoffs', 'Losing_season'],
      dtype='object')

In [38]:
advanced_stats = advanced_stats[['Season', 'Player', 'Tm', 'PER', 'TS%', '3PAr',
       'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%',
       'OWS', 'DWS', 'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP']]

In [47]:
full_players = players_team.merge(advanced_stats, on = ['Player','Tm','Season'],how = 'inner')
full_players.shape

(11860, 61)

In [48]:
full_players.head()

,Season,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,TOV%,USG%,OWS,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,0.0,530.0,103.0,273.0,...,6.4,25.1,-0.5,0.2,-0.3,-0.025,-3.2,-1.5,-4.7,-0.4
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,16.0,959.0,144.0,357.0,...,13.6,22.1,-0.8,0.6,-0.2,-0.008,-3.6,-1.2,-4.8,-0.7
2,1998,Terry Dehere,PG,26.0,SAC,77.0,18.0,1410.0,180.0,451.0,...,16.3,18.6,0.3,0.7,1.0,0.035,-2.5,-0.9,-3.4,-0.5
3,1998,Lawrence Funderburke,PF,27.0,SAC,52.0,1.0,1094.0,191.0,390.0,...,11.8,21.3,1.4,0.7,2.1,0.092,0.9,-1.4,-0.5,0.4
4,1998,Derek Grimm,SF,23.0,SAC,9.0,0.0,34.0,4.0,14.0,...,16.8,23.4,-0.1,0.1,0.0,-0.012,-4.1,1.6,-2.5,0.0


In [55]:
full_players_mvp = full_players.merge(mvp_voting,on = ['Season','Player'],how = 'left')
full_players_mvp.shape

(11860, 64)

In [61]:
full_players_mvp

,Season,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Pts Won,Pts Max,Share
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,0.0,530.0,103.0,273.0,...,0.2,-0.3,-0.025,-3.2,-1.5,-4.7,-0.4,NaN,NaN,NaN
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,16.0,959.0,144.0,357.0,...,0.6,-0.2,-0.008,-3.6,-1.2,-4.8,-0.7,NaN,NaN,NaN
2,1998,Terry Dehere,PG,26.0,SAC,77.0,18.0,1410.0,180.0,451.0,...,0.7,1.0,0.035,-2.5,-0.9,-3.4,-0.5,NaN,NaN,NaN
3,1998,Lawrence Funderburke,PF,27.0,SAC,52.0,1.0,1094.0,191.0,390.0,...,0.7,2.1,0.092,0.9,-1.4,-0.5,0.4,NaN,NaN,NaN
4,1998,Derek Grimm,SF,23.0,SAC,9.0,0.0,34.0,4.0,14.0,...,0.1,0.0,-0.012,-4.1,1.6,-2.5,0.0,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11855,2022,Trevelin Queen,SG,24.0,HOU,10.0,0.0,74.0,15.0,33.0,...,0.1,0.1,0.08,2.7,-0.8,1.9,0.1,NaN,NaN,NaN
11856,2022,Dennis Schröder,PG,28.0,HOU,64.0,29.0,1837.0,312.0,724.0,...,1.9,2.9,0.076,-1.5,-0.4,-1.9,0.0,NaN,NaN,NaN
11857,2022,Alperen Şengün,C,19.0,HOU,72.0,13.0,1489.0,249.0,525.0,...,1.3,2.1,0.068,-1.6,0.6,-1.0,0.4,NaN,NaN,NaN
11858,2022,Jae'Sean Tate,PF,26.0,HOU,78.0,77.0,2056.0,363.0,729.0,...,1.1,3.0,0.071,-1.3,-0.5,-1.8,0.1,NaN,NaN,NaN


In [63]:
full_players_mvp[~full_players_mvp['Share'].isnull()]

,Season,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,DWS,WS,WS/48,OBPM,DBPM,BPM,VORP,Pts Won,Pts Max,Share
9,1998,Mitch Richmond,SG,32.0,SAC,70.0,70.0,2569.0,543.0,1220.0,...,1.3,7.9,0.148,4.4,-0.8,3.6,3.6,3.0,1160.0,0.003
104,1998,Antoine Walker,PF,21.0,BOS,82.0,82.0,3268.0,722.0,1705.0,...,4.0,4.7,0.07,1.6,-1.0,0.6,2.2,6.0,1160.0,0.005
128,1998,Karl Malone,PF,34.0,UTA,81.0,81.0,3030.0,780.0,1472.0,...,4.3,16.4,0.259,6.6,0.7,7.3,7.1,842.0,1160.0,0.726
132,1998,John Stockton,PG,35.0,UTA,64.0,64.0,1858.0,270.0,511.0,...,1.6,8.0,0.206,5.5,0.2,5.6,3.6,5.0,1160.0,0.004
139,1998,Michael Finley,SF,24.0,DAL,82.0,82.0,3394.0,675.0,1505.0,...,2.0,7.5,0.106,3.0,-0.6,2.5,3.8,1.0,1160.0,0.001
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11663,2022,Nikola Jokić,C,26.0,DEN,74.0,74.0,2476.0,764.0,1311.0,...,4.5,15.2,0.296,9.2,4.5,13.7,9.8,875.0,1000.0,0.875
11673,2022,Joel Embiid,C,27.0,PHI,68.0,68.0,2297.0,666.0,1334.0,...,4.1,12.0,0.252,7.2,2.0,9.2,6.5,706.0,1000.0,0.706
11773,2022,Luka Dončić,PG,22.0,DAL,65.0,65.0,2301.0,641.0,1403.0,...,3.8,7.6,0.159,6.4,1.8,8.2,5.9,146.0,1000.0,0.146
11789,2022,Stephen Curry,PG,33.0,GSW,64.0,64.0,2211.0,535.0,1224.0,...,3.4,8.0,0.173,5.4,0.4,5.8,4.4,4.0,1000.0,0.004


In [64]:
full_players_awards = full_players_mvp.merge(dpoy_voting,on = ['Season','Player'],how = 'left',suffixes = ('_mvp','_dpoy'))
full_players_awards.shape

(11860, 67)

In [68]:
full_players_awards.fillna(0,inplace = True)

In [69]:
full_players_awards.to_csv('full_players_dataset.csv',index = False)

In [99]:
full_dataset = pd.read_csv('full_players_dataset.csv')
full_dataset.head()

,Season,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,OBPM,DBPM,BPM,VORP,Pts Won_mvp,Pts Max_mvp,Share_mvp,Pts Won_dpoy,Pts Max_dpoy,Share_dpoy
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,0.0,530.0,103.0,273.0,...,-3.2,-1.5,-4.7,-0.4,0.0,0.0,0.0,0.0,0.0,0.0
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,16.0,959.0,144.0,357.0,...,-3.6,-1.2,-4.8,-0.7,0.0,0.0,0.0,0.0,0.0,0.0
2,1998,Terry Dehere,PG,26.0,SAC,77.0,18.0,1410.0,180.0,451.0,...,-2.5,-0.9,-3.4,-0.5,0.0,0.0,0.0,0.0,0.0,0.0
3,1998,Lawrence Funderburke,PF,27.0,SAC,52.0,1.0,1094.0,191.0,390.0,...,0.9,-1.4,-0.5,0.4,0.0,0.0,0.0,0.0,0.0,0.0
4,1998,Derek Grimm,SF,23.0,SAC,9.0,0.0,34.0,4.0,14.0,...,-4.1,1.6,-2.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [89]:
full_dataset.columns

Index(['Season', 'Player', 'Pos', 'Age', 'Tm', 'G', 'GS', 'MP', 'FG', 'FGA',
       'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'Year', 'Team', 'W', 'L', 'W/L%', 'GB', 'PS/G', 'PA/G', 'SRS',
       'Playoffs', 'Losing_season', 'PER', 'TS%', '3PAr', 'FTr', 'ORB%',
       'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%', 'OWS', 'DWS',
       'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Pts Won_mvp',
       'Pts Max_mvp', 'Share_mvp', 'Pts Won_dpoy', 'Pts Max_dpoy',
       'Share_dpoy'],
      dtype='object')

In [100]:
full_dataset['received_mvp_votes'] = [1 if row['Share_mvp']!=0 else 0 for idx,row in full_dataset.iterrows()]
full_dataset['received_dpoy_votes'] = [1 if row['Share_dpoy']!=0 else 0 for idx,row in full_dataset.iterrows()]

In [101]:
cols_to_normalize = ['G','GS','W','L','FG', 'FGA', 'FG%', '3P', '3PA', '3P%', '2P', '2PA', '2P%', 'eFG%', 'FT', 'FTA',
       'FT%', 'ORB', 'DRB', 'TRB', 'AST', 'STL', 'BLK', 'TOV', 'PF', 'PTS',
       'PER', 'TS%', '3PAr', 'FTr', 'ORB%', 'DRB%', 'TRB%', 'AST%', 'STL%', 'BLK%', 'TOV%', 'USG%']

In [102]:
normalized = full_dataset.groupby('Season')[cols_to_normalize].transform(lambda x: (x - x.mean()) / x.std())

In [103]:
full_normalized = full_dataset.merge(normalized,right_index = True,left_index=True,suffixes = ('','_normalized'))

In [104]:
full_normalized.head()

,Season,Player,Pos,Age,Tm,G,GS,MP,FG,FGA,...,3PAr_normalized,FTr_normalized,ORB%_normalized,DRB%_normalized,TRB%_normalized,AST%_normalized,STL%_normalized,BLK%_normalized,TOV%_normalized,USG%_normalized
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,31.0,0.0,530.0,103.0,273.0,...,-0.223145,-1.479163,-1.061591,-1.082926,-1.256266,0.678781,-0.270151,-0.824493,-1.271023,1.028959
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,59.0,16.0,959.0,144.0,357.0,...,-0.577654,0.059612,-0.333466,-0.775932,-0.686891,-0.475951,0.010661,-0.352740,-0.260471,0.499256
2,1998,Terry Dehere,PG,26.0,SAC,77.0,18.0,1410.0,180.0,451.0,...,0.817135,-0.627813,-0.984946,-1.050611,-1.195261,0.967465,0.010661,-0.765524,0.118486,-0.118731
3,1998,Lawrence Funderburke,PF,27.0,SAC,52.0,1.0,1094.0,191.0,390.0,...,-0.781061,0.403325,0.241370,0.419729,0.370520,-0.358339,-0.925378,-0.352740,-0.513109,0.358002
4,1998,Derek Grimm,SF,23.0,SAC,9.0,0.0,34.0,4.0,14.0,...,4.094891,-1.034981,-1.291525,-0.000368,-0.727560,-1.427536,2.537968,0.295920,0.188663,0.728794


In [105]:
full_normalized.drop(cols_to_normalize,axis = 1,inplace=True)

In [95]:
full_normalized.head()

,Season,Player,Pos,Age,Tm,MP,Year,Team,W,L,...,3PAr_normalized,FTr_normalized,ORB%_normalized,DRB%_normalized,TRB%_normalized,AST%_normalized,STL%_normalized,BLK%_normalized,TOV%_normalized,USG%_normalized
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,530.0,1998,SAC,27,55,...,-0.223145,-1.479163,-1.061591,-1.082926,-1.256266,0.678781,-0.270151,-0.824493,-1.271023,1.028959
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,959.0,1998,SAC,27,55,...,-0.577654,0.059612,-0.333466,-0.775932,-0.686891,-0.475951,0.010661,-0.352740,-0.260471,0.499256
2,1998,Terry Dehere,PG,26.0,SAC,1410.0,1998,SAC,27,55,...,0.817135,-0.627813,-0.984946,-1.050611,-1.195261,0.967465,0.010661,-0.765524,0.118486,-0.118731
3,1998,Lawrence Funderburke,PF,27.0,SAC,1094.0,1998,SAC,27,55,...,-0.781061,0.403325,0.241370,0.419729,0.370520,-0.358339,-0.925378,-0.352740,-0.513109,0.358002
4,1998,Derek Grimm,SF,23.0,SAC,34.0,1998,SAC,27,55,...,4.094891,-1.034981,-1.291525,-0.000368,-0.727560,-1.427536,2.537968,0.295920,0.188663,0.728794


In [106]:
full_normalized.to_csv("normalized_data.csv",index=False)

## Load Cleaned Data

In [107]:
full_dataset = pd.read_csv('normalized_data.csv')
full_dataset.head()

,Season,Player,Pos,Age,Tm,MP,Year,Team,W/L%,GB,...,3PAr_normalized,FTr_normalized,ORB%_normalized,DRB%_normalized,TRB%_normalized,AST%_normalized,STL%_normalized,BLK%_normalized,TOV%_normalized,USG%_normalized
0,1998,Mahmoud Abdul-Rauf,PG,28.0,SAC,530.0,1998,SAC,0.329,34.0,...,-0.223145,-1.479163,-1.061591,-1.082926,-1.256266,0.678781,-0.270151,-0.824493,-1.271023,1.028959
1,1998,Tariq Abdul-Wahad,SG,23.0,SAC,959.0,1998,SAC,0.329,34.0,...,-0.577654,0.059612,-0.333466,-0.775932,-0.686891,-0.475951,0.010661,-0.352740,-0.260471,0.499256
2,1998,Terry Dehere,PG,26.0,SAC,1410.0,1998,SAC,0.329,34.0,...,0.817135,-0.627813,-0.984946,-1.050611,-1.195261,0.967465,0.010661,-0.765524,0.118486,-0.118731
3,1998,Lawrence Funderburke,PF,27.0,SAC,1094.0,1998,SAC,0.329,34.0,...,-0.781061,0.403325,0.241370,0.419729,0.370520,-0.358339,-0.925378,-0.352740,-0.513109,0.358002
4,1998,Derek Grimm,SF,23.0,SAC,34.0,1998,SAC,0.329,34.0,...,4.094891,-1.034981,-1.291525,-0.000368,-0.727560,-1.427536,2.537968,0.295920,0.188663,0.728794


## Feature Selection

In [108]:
full_dataset.columns

Index(['Season', 'Player', 'Pos', 'Age', 'Tm', 'MP', 'Year', 'Team', 'W/L%',
       'GB', 'PS/G', 'PA/G', 'SRS', 'Playoffs', 'Losing_season', 'OWS', 'DWS',
       'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'Pts Won_mvp',
       'Pts Max_mvp', 'Share_mvp', 'Pts Won_dpoy', 'Pts Max_dpoy',
       'Share_dpoy', 'received_mvp_votes', 'received_dpoy_votes',
       'G_normalized', 'GS_normalized', 'W_normalized', 'L_normalized',
       'FG_normalized', 'FGA_normalized', 'FG%_normalized', '3P_normalized',
       '3PA_normalized', '3P%_normalized', '2P_normalized', '2PA_normalized',
       '2P%_normalized', 'eFG%_normalized', 'FT_normalized', 'FTA_normalized',
       'FT%_normalized', 'ORB_normalized', 'DRB_normalized', 'TRB_normalized',
       'AST_normalized', 'STL_normalized', 'BLK_normalized', 'TOV_normalized',
       'PF_normalized', 'PTS_normalized', 'PER_normalized', 'TS%_normalized',
       '3PAr_normalized', 'FTr_normalized', 'ORB%_normalized',
       'DRB%_normalized', 'TRB%_normal

## Create Models

### Random Forest

#### MVP

In [137]:
mvp_dataset = full_dataset[['Playoffs', 'OWS', 'DWS',
       'WS', 'WS/48', 'OBPM', 'DBPM', 'BPM', 'VORP', 'received_mvp_votes',
       'G_normalized', 'GS_normalized', 'W_normalized', 'L_normalized',
       'FG_normalized', 'FGA_normalized', 'FG%_normalized', '3P_normalized',
       '3PA_normalized', '3P%_normalized', '2P_normalized', '2PA_normalized',
       '2P%_normalized', 'eFG%_normalized', 'FT_normalized', 'FTA_normalized',
       'FT%_normalized', 'ORB_normalized', 'DRB_normalized', 'TRB_normalized',
       'AST_normalized', 'STL_normalized', 'BLK_normalized', 'TOV_normalized',
       'PF_normalized', 'PTS_normalized', 'PER_normalized', 'TS%_normalized',
       '3PAr_normalized', 'FTr_normalized', 'ORB%_normalized',
       'DRB%_normalized', 'TRB%_normalized', 'AST%_normalized',
       'STL%_normalized', 'BLK%_normalized', 'TOV%_normalized',
       'USG%_normalized']]

In [138]:
Xm = np.array(mvp_dataset.drop(['received_mvp_votes'],axis=1))
ym = np.array(mvp_dataset['received_mvp_votes'])

Xm_train, Xm_test, ym_train, ym_test = train_test_split(Xm, ym, test_size=0.1, random_state=42)

In [146]:
mvp_clf = RandomForestClassifier(30, oob_score = True).fit(Xm_train,ym_train)

#### DPOY

In [140]:
dpoy_dataset = full_dataset[['Playoffs', 'DWS',
       'WS', 'WS/48', 'DBPM', 'VORP', 'received_dpoy_votes',
       'G_normalized', 'GS_normalized', 'W_normalized', 'L_normalized',
       'ORB_normalized', 'DRB_normalized', 'TRB_normalized',
       'STL_normalized', 'BLK_normalized',
       'PF_normalized', 'PTS_normalized', 'PER_normalized',
       'DRB%_normalized', 'TRB%_normalized',
       'STL%_normalized', 'BLK%_normalized']]

In [141]:
Xd = np.array(dpoy_dataset.drop(['received_dpoy_votes'],axis=1))
yd = np.array(dpoy_dataset['received_dpoy_votes'])

Xd_train, Xd_test, yd_train, yd_test = train_test_split(Xd, yd, test_size=0.1, random_state=42)

In [142]:
dpoy_clf = RandomForestClassifier(30, oob_score = True).fit(Xd_train,yd_train)

### Regression

## Evaluate Models

### Random Forest

### MVP

In [147]:
print(f'The training accuracy: {mvp_clf.score(Xm_train,ym_train)}')
print(f'The out of bag classification accuracy: {mvp_clf.oob_score_}')

The training accuracy: 0.9994378864530635
The out of bag classification accuracy: 0.9835113359565298


In [148]:
mvp_clf.score(Xm_test,ym_test)

0.9881956155143339

In [158]:
for i in range(20,100,5):
    mvp_clf = RandomForestClassifier(i, oob_score = True).fit(Xm_train,ym_train)
    print(f'Test Accuracy with {i} trees = {mvp_clf.score(Xm_test,ym_test)}')

Test Accuracy with 20 trees = 0.9856661045531198
Test Accuracy with 25 trees = 0.9873524451939292
Test Accuracy with 30 trees = 0.9890387858347386
Test Accuracy with 35 trees = 0.9873524451939292
Test Accuracy with 40 trees = 0.9873524451939292
Test Accuracy with 45 trees = 0.9881956155143339
Test Accuracy with 50 trees = 0.9890387858347386
Test Accuracy with 55 trees = 0.9898819561551433
Test Accuracy with 60 trees = 0.9890387858347386
Test Accuracy with 65 trees = 0.9881956155143339
Test Accuracy with 70 trees = 0.9873524451939292
Test Accuracy with 75 trees = 0.9890387858347386
Test Accuracy with 80 trees = 0.9907251264755481
Test Accuracy with 85 trees = 0.9881956155143339
Test Accuracy with 90 trees = 0.9873524451939292
Test Accuracy with 95 trees = 0.9898819561551433


In [167]:
i = 0
feat_imp = {}
cols = list(mvp_dataset.columns)
for val in mvp_clf.feature_importances_:
    if cols[i] == 'received_mvp_votes':
        i += 1
        pass
    feat_imp[cols[i]] = val
    print(f'The imporance of {cols[i]} = {val}')
    i += 1

The imporance of Playoffs = 0.006048126467235637
The imporance of OWS = 0.0539764882458436
The imporance of DWS = 0.015831658628647816
The imporance of WS = 0.06759978902393325
The imporance of WS/48 = 0.02603269566874107
The imporance of OBPM = 0.02567729568784092
The imporance of DBPM = 0.01504052285547773
The imporance of BPM = 0.06742930822007782
The imporance of VORP = 0.07872809106664343
The imporance of G_normalized = 0.008823516289855834
The imporance of GS_normalized = 0.01030756241188816
The imporance of W_normalized = 0.027488399137227983
The imporance of L_normalized = 0.029105821876367788
The imporance of FG_normalized = 0.05761399966367629
The imporance of FGA_normalized = 0.0236655120464544
The imporance of FG%_normalized = 0.006631221486345075
The imporance of 3P_normalized = 0.007538648739092598
The imporance of 3PA_normalized = 0.007121695726938395
The imporance of 3P%_normalized = 0.006370551460039463
The imporance of 2P_normalized = 0.020545181394127685
The imporanc

#### DPOY

In [149]:
print(f'The training accuracy: {dpoy_clf.score(Xd_train,yd_train)}')
print(f'The out of bag classification accuracy: {dpoy_clf.oob_score_}')

The training accuracy: 0.9996252576353757
The out of bag classification accuracy: 0.9728311785647368


In [151]:
dpoy_clf.score(Xd_test,yd_test)

0.9704890387858347

In [152]:
dpoy_clf.feature_importances_

array([0.00336575, 0.11301797, 0.05972705, 0.03280648, 0.08040351,
       0.04352707, 0.03180111, 0.03844077, 0.03373632, 0.02811054,
       0.04112855, 0.04540292, 0.03898762, 0.06209132, 0.08296658,
       0.03459083, 0.0293083 , 0.03980603, 0.03826666, 0.02989093,
       0.04685449, 0.04576919])

In [159]:
for i in range(20,100,5):
    dpoy_clf = RandomForestClassifier(i, oob_score = True).fit(Xd_train,yd_train)
    print(f'Test Accuracy with {i} trees = {dpoy_clf.score(Xd_test,yd_test)}')

/Users/claykindiger/anaconda3/lib/python3.11/site-packages/sklearn/ensemble/_forest.py:578: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable OOB estimates.
  warn(


Test Accuracy with 20 trees = 0.9721753794266442
Test Accuracy with 25 trees = 0.9721753794266442
Test Accuracy with 30 trees = 0.9730185497470489
Test Accuracy with 35 trees = 0.9721753794266442
Test Accuracy with 40 trees = 0.9721753794266442
Test Accuracy with 45 trees = 0.9713322091062394
Test Accuracy with 50 trees = 0.9730185497470489
Test Accuracy with 55 trees = 0.9721753794266442
Test Accuracy with 60 trees = 0.9713322091062394
Test Accuracy with 65 trees = 0.9713322091062394
Test Accuracy with 70 trees = 0.9730185497470489
Test Accuracy with 75 trees = 0.9721753794266442
Test Accuracy with 80 trees = 0.9721753794266442
Test Accuracy with 85 trees = 0.9713322091062394
Test Accuracy with 90 trees = 0.9738617200674536
Test Accuracy with 95 trees = 0.9730185497470489


In [157]:
i = 0
cols = list(dpoy_dataset.columns)
for val in dpoy_clf.feature_importances_:
    if cols[i] == 'received_dpoy_votes':
        i += 1
        pass
    print(f'The imporance of {cols[i]} = {val}')
    i += 1

The imporance of Playoffs = 0.0033657466127611597
The imporance of DWS = 0.11301796522739042
The imporance of WS = 0.05972705028653594
The imporance of WS/48 = 0.032806481163311106
The imporance of DBPM = 0.08040351136247544
The imporance of VORP = 0.043527066593806514
The imporance of G_normalized = 0.031801111670449675
The imporance of GS_normalized = 0.03844077008073614
The imporance of W_normalized = 0.0337363163355385
The imporance of L_normalized = 0.02811054254033162
The imporance of ORB_normalized = 0.041128546345689736
The imporance of DRB_normalized = 0.045402921842588394
The imporance of TRB_normalized = 0.03898762460810132
The imporance of STL_normalized = 0.06209131947655253
The imporance of BLK_normalized = 0.0829665829342641
The imporance of PF_normalized = 0.03459083362458792
The imporance of PTS_normalized = 0.0293083045178513
The imporance of PER_normalized = 0.03980603039699095
The imporance of DRB%_normalized = 0.038266663767311804
The imporance of TRB%_normalized =

### Regression